In [17]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import pickle as pkl

%run ../../scripts/mfss_model_train.py
%run ../../scripts/coherence_analysis_and_transform.py
%run ../../scripts/utils/data_processing.py
%matplotlib inline

# Load in training and test data

In [3]:
data_path = '../../data/processed/'

In [8]:
# Read in the Hannum dataset for training
hannum = pd.read_pickle(data_path + 'GSE40279_hannum/hannum32.pkl')
hannum_meta = pd.read_excel(data_path + 'GSE40279_hannum/hannum_pmeta.xlsx')

In [4]:
art = np.load(data_path + 'GSE42861_arthritis/arthritis32_full.npy', allow_pickle=True)
art_meta = pd.read_excel(data_path + 'GSE42861_arthritis/arthritis_pmeta.xlsx')
cgs=list_reader(data_path + 'GSE42861_arthritis/cgs_full.txt')
art = pd.DataFrame(art, columns=cgs)

In [9]:
# Read in the Lehne datasat for testing
lehne = np.load(data_path + 'GSE55763_lehne/lehne32_reduced.npy')
lehne_meta = pd.read_excel(data_path + 'GSE55763_lehne/lehne_pmeta.xlsx')
lehne_cgs = list_reader(data_path + 'GSE55763_lehne/cgs')
lehne = pd.DataFrame(lehne, columns=lehne_cgs)

### Read in the filtered (less SNPs and CpGs missing from GSE55763) age correlations 'filtered_age_corrs.xlsx' stored in 'cpg_sets' directory

In [5]:
age_corrs=pd.read_excel(data_path + 'cpg_sets/filtered_age_corrs.xlsx')

In [18]:
#Identify arthritis-shifted CpGs
up_shift, down_shift = get_shifts(art, art_meta, age_corrs.CpG[:10000], 'Control')

#Combine
d_shift = pd.concat([up_shift,down_shift])
d_shift.reset_index(inplace=True, drop=True)

In [19]:
#Get correlation magnitudes, sort correlation magnitude (descending)
d_shift['Abs. correlation'] = abs(d_shift.Correlation)
d_shift.sort_values(by='Abs. correlation', ignore_index=True, inplace=True,ascending=False)
d_shift['Abs. d'] = abs(d_shift['Cohens d'])

# Generate Supplementary Table 3

In [20]:
d_shift

,CpG,Shift,Correlation,Cohens d,Sign,Order,Abs. correlation,Abs. d
0,cg16867657,-0.009140,0.847155,0.663105,-1,0,0.847155,0.663105
1,cg21572722,-0.013792,0.745844,1.246969,-1,6,0.745844,1.246969
2,cg22454769,0.009697,0.730099,-0.629524,1,3,0.730099,0.629524
3,cg06639320,-0.007488,0.720896,0.569131,-1,1,0.720896,0.569131
4,cg08097417,-0.007874,0.708310,0.739369,-1,23,0.708310,0.739369
...,...,...,...,...,...,...,...,...
9995,cg16456337,0.011331,0.000061,-1.068008,1,8655,0.000061,1.068008
9996,cg20709868,0.016851,-0.000055,-1.393279,1,8897,0.000055,1.393279
9997,cg16241861,0.011039,0.000055,-0.737081,1,6492,0.000055,0.737081
9998,cg01585758,-0.002377,0.000037,0.499104,-1,4931,0.000037,0.499104


In [10]:

# #Re-sort by disease shift magnitude
# d_shift['Abs. shift'] = abs(d_shift.Shift)
# d_shift.sort_values(by="Abs. shift", inplace=True,ignore_index=True,ascending=False)